<a href="https://colab.research.google.com/github/Shivanipappu/FMML-LAB-ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#ANSWER 1

Yes, averaging the validation accuracy across multiple splits of your data can often give more consistent and reliable results compared to relying on a single validation split. This technique is commonly known as cross-validation, and it helps to reduce the impact of data randomness and provide a more robust estimate of your model's performance.

Here's why averaging validation accuracy across multiple splits is beneficial:

1.Reduced Variance: A single validation split might be unrepresentative of your entire dataset, especially if your data is not evenly distributed or contains outliers. Averaging over multiple splits helps to reduce the variance introduced by such randomness.

2.Better Generalization Estimate: By training and validating your model on different subsets of the data, you obtain a more accurate estimate of how well your model will generalize to unseen data. This is particularly useful when you have limited data and want to assess your model's true performance.

3.Detecting Overfitting: Cross-validation can help you detect overfitting. If your model performs significantly better on the training data compared to the validation data in multiple splits, it's a sign that your model might be overfitting.

Common types of cross-validation include k-fold cross-validation and stratified k-fold cross-validation. In k-fold cross-validation, you split your dataset into k equally sized subsets (folds). You then train and validate your model k times, each time using a different fold as the validation set and the remaining folds as the training set. The results are then averaged to obtain a more stable performance metric.

However, it's important to note that cross-validation can be computationally expensive, especially if you have a large dataset or complex models. In some cases, a single validation split or a train-validation-test split may be sufficient, but using cross-validation is generally recommended for obtaining more consistent and trustworthy results, especially when dealing with limited data or when model performance evaluation is critical.

#ANSWER 2

Cross-validation, by itself, does not provide a more accurate estimate of the test accuracy. Test accuracy is only determined by evaluating your model on a completely independent and unseen test dataset. Cross-validation is primarily used to estimate how well your model is likely to perform on new, unseen data based on the available training data.

Here's how cross-validation and test accuracy estimation are related:

1.Cross-Validation for Model Selection and Hyperparameter Tuning: Cross-validation is often used during the model development and tuning process. It helps you assess how well your model generalizes to different subsets of your training data and aids in selecting the best model architecture and hyperparameters. It provides a more reliable estimate of how your model might perform on unseen data compared to a single validation split.

2.Test Dataset for Final Evaluation: Once you have selected your model and tuned your hyperparameters using cross-validation, you should use a separate, previously unseen test dataset to obtain an accurate estimate of how your model will perform in a real-world scenario. This test dataset should not have been used during training or cross-validation. The performance metrics obtained on the test dataset, including test accuracy, reflect how your model is expected to perform in production or when applied to new, unseen data.

In summary, cross-validation helps you select the best model and assess its generalization performance based on the available training data, but the test accuracy, which is the most accurate estimate of your model's real-world performance, can only be determined by evaluating your chosen model on a separate and independent test dataset that has not been used during model development or hyperparameter tuning.






#ANSWER 3

ChatGPT
The number of iterations in machine learning and deep learning training can have a significant impact on the estimate of model performance, but the relationship between the number of iterations and the quality of the estimate is not necessarily straightforward. It depends on several factors:

1.Underfitting and Overfitting: The number of iterations affects the trade-off between underfitting and overfitting. With too few iterations, your model may underfit the training data, leading to poor generalization. With too many iterations, your model may overfit the training data, capturing noise and performing poorly on new data.

2.Convergence: In many cases, increasing the number of iterations can lead to better convergence of your model. This means that your model becomes better at minimizing the training loss and, as a result, may improve its ability to generalize to new data. However, there is a point of diminishing returns, where further iterations may not significantly improve performance.

3.Computational Resources: Training deep learning models with a large number of iterations can be computationally expensive and time-consuming. You need to consider your available computational resources when deciding on the number of iterations.

4.Early Stopping: To find an optimal number of iterations, it's common to use techniques like early stopping. Early stopping monitors the model's performance on a validation set during training and stops training when performance begins to degrade. This helps prevent overfitting and can lead to better estimates of model performance.

5.Validation and Cross-Validation: The choice of the number of iterations can also impact the results of validation or cross-validation. If you terminate training too early, your model may not have had a chance to converge to its best performance. If you train for too many iterations, you risk overfitting to the validation data.



#ANSWER 4

Increasing the number of iterations (or epochs) is one way to compensate for having a very small training dataset or validation dataset, but it's not a perfect solution, and there are limitations to what can be achieved by simply increasing iterations. Here are some considerations:

Advantages of Increasing Iterations:

1.Extraction of More Information: With more iterations, your model has more opportunities to see the same data multiple times, potentially allowing it to extract more information from the limited training data available.

2.Better Model Convergence: In some cases, increasing iterations can help your model converge to a lower training loss, which may indicate better utilization of the training data.

Limitations and Concerns:

1.Overfitting: Increasing iterations can make your model more prone to overfitting, especially when you have a small dataset. The model may start memorizing the training examples rather than learning meaningful patterns, leading to poor generalization to new data.

2.Noisy Estimation: If you have a very small validation dataset, estimating model performance becomes more challenging. While more iterations may lead to better training performance, it doesn't necessarily mean your model will generalize well to new data.

3.Computational Resources: Training for a large number of iterations can be computationally expensive, and it may not be feasible if you have limited resources or time constraints.

Possible Solutions:

1.Data Augmentation: If your training dataset is small, you can augment it by generating new training examples through transformations (e.g., rotations, flips, cropping) of the existing data. Data augmentation can effectively increase the effective size of your training dataset.

2.Regularization Techniques: Use regularization techniques such as dropout, weight decay, or early stopping to mitigate overfitting when increasing iterations. Early stopping, in particular, can be effective in preventing the model from training for too long and overfitting the data.

3.Transfer Learning: If you have a small dataset, consider using pre-trained models as a starting point. Fine-tuning a pre-trained model on your specific task can often lead to better results with fewer iterations.

4.Cross-Validation: If you have a small dataset and are concerned about the reliability of your performance estimates, consider using cross-validation. This technique allows you to assess how well your model generalizes to different subsets of your small dataset.


